Hi Jan, while searching for the bug for my PQK kernel DE, I came across something strange. I dont know if it is a matter of different definitions or something else. 


If one applies an $R_x(x)$ rotation in a circuit and calculates the expectation value with regards to $X,Y$ and $Z$. One finds that (see derivation below):

$<X \rho> =  0$ 


$<Y\rho> = sin(x)$ 

$<Z\rho> = cos(x)$ 


If one uses squlearn, one finds that the sign with regards to Y is changed:

$<X \rho> =  0$ 


$<Y\rho> = -sin(x)$ 

$<Z\rho> = cos(x)$  

Do you have an explanation why?

Derivation using sympy

In [1]:
import sympy as sp
#also https://arxiv.org/pdf/2206.06686 equation 3 and 4
x = sp.symbols("x", real=True)
RX = sp.Matrix([[sp.cos(x/2), 1j*sp.sin(x/2)], [1j*sp.sin(x/2), sp.cos(x/2)]])

X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -1j], [1j, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])

ket0 = sp.Matrix([[1], [0]])
bra0 = ket0.T

rhox = RX @ ket0 @ bra0 @ RX.H

value = 0.12
print(f"f(x) = {sp.trace(X@rhox).simplify()}, {sp.trace(Y@rhox).simplify()}, {sp.trace(Z@rhox).simplify()}")
print("f(0.5)", [f.subs(x, value).evalf() for f in [sp.trace(X@rhox), sp.trace(Y@rhox), sp.trace(Z@rhox)]])


f(x) = 0, 1.0*sin(x), 1.0*cos(x)
f(0.5) [0, 0.119712207288919, 0.992808635853866]


Experiment squlearn

In [1]:
import numpy as np
from squlearn.observables import  SinglePauli
from squlearn.encoding_circuit import LayeredEncodingCircuit
from squlearn.qnn.lowlevel_qnn import LowLevelQNN
from squlearn import Executor

def Separable_rx(num_qubits, num_layers):
    """
    Separable_rx(num_qubits, num_layers)
    Returns a circuit that is similar to the one used in IQP.
    """
    fmap = LayeredEncodingCircuit(num_qubits=num_qubits, num_features=num_qubits)
    for layer in range(num_layers):
        fmap.Rx("x")
    return fmap

num_qubits = 1
x_array = np.array([[value]])

circuit = Separable_rx(1, 1)
observable = [SinglePauli(num_qubits, 0, "X"), SinglePauli(num_qubits, 0, "Y"), SinglePauli(num_qubits, 0, "Z")]
qnn_pennylane_statevector = LowLevelQNN(circuit, observable, Executor("pennylane"))

param = []
param_obs = []

print("Pennylane statevector ")
print("f\n", qnn_pennylane_statevector.evaluate(x_array, param, param_obs, "f")["f"])

NameError: name 'value' is not defined

In [1]:

"""Tests for QKRR"""

import pytest
import numpy as np
import sympy as sp
from unittest.mock import MagicMock

from sklearn.datasets import make_regression
from sklearn.exceptions import NotFittedError
from sklearn.preprocessing import MinMaxScaler

from squlearn import Executor
from squlearn.encoding_circuit import ParamZFeatureMap, LayeredEncodingCircuit
from squlearn.kernel import QKRR
from squlearn.kernel.matrix import ProjectedQuantumKernel, FidelityKernel


class TestProjectedQuantumKernel:
    def test_that_single_variable_derivatives_are_correct(self):
            """
            Test that the single variable derivatives are correct

            By implementing U = U_x(x)U_y(p), we obtain k(x,y, p) = exp(-2.0*gamma*(1 - cos(x - y))*cos(p)**2)
            """
            #Single variable derivatives 

            x,y,gamma_sp, p = sp.symbols("x y gamma p")

            sympy_K = sp.exp(-2*gamma_sp*(1-sp.cos(x-y))*sp.cos(p)**2)


            sympy_dKdx = sp.diff(sympy_K, x)
            sympy_dKdy = sp.diff(sympy_K, y)
            sympy_dKdxdx = sp.diff(sympy_dKdx, x) #I will use this expression, the derivation can be seen in the old notebooks 

            testing_values = 0.79, -0.31
            x_num, y_num = 0.79, -0.31
            p_num = -0.63
            gamma_num = 0.08

            sympy_values = {
                "K": sympy_K.evalf(subs={x: x_num, y: y_num, gamma_sp: gamma_num, p: p_num}),
                "dKdx": sympy_dKdx.evalf(subs={x: x_num, y: y_num, gamma_sp: gamma_num, p: p_num}),
                "dKdy": sympy_dKdy.evalf(subs={x: x_num, y: y_num, gamma_sp: gamma_num, p: p_num}),
                "dKdxdx": sympy_dKdxdx.evalf(subs={x: x_num, y: y_num, gamma_sp: gamma_num, p: p_num}),
            }

            executor = Executor()
            num_features = 1
            encoding_circuit = LayeredEncodingCircuit(
                num_qubits=num_features, num_features=num_features
            )
            encoding_circuit.Ry("p")
            encoding_circuit.Rx("x")

            kernel = ProjectedQuantumKernel(
                encoding_circuit=encoding_circuit, executor=executor, outer_kernel="gaussian", gamma=gamma_num, initial_parameters=np.array([p_num])
            )

            values = kernel.evaluate_derivatives([x_num], [y_num], ["K", "dKdx", "dKdy", "dKdxdx", "dKdp"])
            print(values)
            
            for key in ["K", "dKdx", "dKdy", "dKdxdx", "dKdp"]:
                assert np.isclose(float(values[key][0][0]), float(sympy_values[key]), atol=1e-7)
    def test_that_multi_variable_derivatives_are_correct(self):
            """
            Test that the single variable derivatives are correct

            By implementing U = U_x(x)U_y(p), we obtain k(x,y, p) = exp(-2.0*gamma*(1 - cos(x - y))*cos(p)**2)
            """
            #Single variable derivatives 

            x0,y0,gamma_sp, p0 = sp.symbols("x0 y0 gamma p0")
            x1,y1,gamma_sp, p1 = sp.symbols("x1 y1 gamma p1")
            

            #sp.exp(-gamma*(-2.0*cos(x0 - y0) - 2.0*cos(x1 - y1) + 4.0)*cos(p0)**2)
            sympy_K = sp.exp(-2.0*gamma_sp*(-sp.cos(p0)**2*sp.cos(x0 - y0) + sp.cos(p0)**2 - sp.cos(p1)**2*sp.cos(x1 - y1) + sp.cos(p1)**2))

            sympy_dKdx0 = sp.diff(sympy_K, x0)
            sympy_dKdy0 = sp.diff(sympy_K, y0)
            sympy_dKdp0 = sp.diff(sympy_K, p0)

            x0_num, y0_num = 0.79, -0.31
            x1_num, y1_num = 0.9, -1.31
            p_num = -0.63
            gamma_num = 0.08
            
            sympy_values = {
                "K": sympy_K.evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num}),
                "dKdx": [sp.diff(sympy_K, x0).evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num}),
                         sp.diff(sympy_K, x1).evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num})],
                "dKdy": [sp.diff(sympy_K, y0).evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num}),
                            sp.diff(sympy_K, y1).evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num})],
                "dKdp": [sp.diff(sympy_K, p0).evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num}),
                            sp.diff(sympy_K, p1).evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num})],
            }

            executor = Executor()
            num_features = 2
            encoding_circuit = LayeredEncodingCircuit(
                num_qubits=num_features, num_features=num_features
            )
            encoding_circuit.Ry("p")
            encoding_circuit.Rx("x")

            kernel = ProjectedQuantumKernel(
                encoding_circuit=encoding_circuit, executor=executor, outer_kernel="gaussian", gamma=gamma_num, initial_parameters=np.array([p_num, p_num])
            )

            values = kernel.evaluate_derivatives([[x0_num, x1_num]], [[y0_num, y1_num]], ["K", "dKdx", "dKdy", "dKdp"]) #"dKdp"
            print(values)
            print(sympy_values)

            #return values
            for key in ["K", "dKdx", "dKdy", "dKdp"]:
                 np.all
                 print(np.allclose(np.array(values[key]).flatten().astype(float), np.array(sympy_values[key]).astype(float), atol=1e-7))
                 #assert np.allclose(np.array(values[key]).flatten(), np.array(sympy_values[key]), atol=1e-7)

In [4]:
ProjectedQuantumKernel?

Init signature:
ProjectedQuantumKernel(
    encoding_circuit: squlearn.encoding_circuit.encoding_circuit_base.EncodingCircuitBase,
    executor: squlearn.util.executor.Executor,
    measurement: Union[str, squlearn.observables.observable_base.ObservableBase, list] = 'XYZ',
    outer_kernel: Union[str, squlearn.kernel.matrix.projected_quantum_kernel.OuterKernelBase] = 'gaussian',
    initial_parameters: Optional[numpy.ndarray] = None,
    parameter_seed: Optional[int] = 0,
    regularization: Optional[str] = None,
    caching: bool = True,
    **kwargs,
) -> None
Docstring:     
Projected Quantum Kernel for Quantum Kernel Algorithms

The Projected Quantum Kernel embeds classical data into a quantum Hilbert space first and
than projects down into a real space by measurements. The real space is than used to
evaluate a classical kernel.

The projection is done by evaluating the expectation values of the encoding circuit with respect
to given Pauli operators. This is achieved by supplying a

In [2]:
executor = Executor()
num_features = 2
encoding_circuit = LayeredEncodingCircuit(
    num_qubits=num_features, num_features=num_features
)
encoding_circuit.Ry("p")
encoding_circuit.Rx("x")

kernel = ProjectedQuantumKernel(
    encoding_circuit=encoding_circuit, executor=executor, outer_kernel="gaussian", gamma=0.7, initial_parameters=np.array([0.25, 0.3]), 
)

values = kernel.evaluate_derivatives([[1, 2.5], [1, 3.5]], [[1, 2.5], [1, 4.5]], ["K", "dKdx", "dKdy", "dKdp", "dKdop"]) #"dKdp"

num_features? (2, 6, 2)


ValueError: cannot reshape array of size 0 into shape (2,1)

In [13]:
values["dKdp"][0]

array([[ 0.00000000e+00,  0.00000000e+00],
       [ 1.08501055e-17, -3.02846234e-17]])

In [2]:
np.array([[[2]]]).flatten()

array([2])

In [17]:
TestProjectedQuantumKernel().test_that_single_variable_derivatives_are_correct()

C:\Users\jsl-rf\AppData\Local\Temp\ipykernel_29308\1562624219.py:62: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  assert np.isclose(float(values[key][0][0]), float(sympy_values[key]), atol=1e-7)


KeyError: 'dKdp'

In [18]:
v = TestProjectedQuantumKernel().test_that_multi_variable_derivatives_are_correct()

num_features? (1, 6, 2)
{'x': [[0.79, 0.9]], 'param': array([-0.63, -0.63]), 'param_op': array([], dtype=float64), 'K': array([[0.79942296]]), 'dKdx': array([[[-0.07442651]],

       [[-0.06702432]]]), 'dKdy': array([[[0.07442651]],

       [[0.06702432]]]), 'dKdp': array([[[-0.06654088]],

       [[-0.19442811]]])}
{'K': 0.799422958081041, 'dKdx': [-0.0744265101662485, -0.0670243158865214], 'dKdy': [0.0744265101662485, 0.0670243158865214], 'dKdp': [-0.0665408787508404, -0.194428110958415]}
True
True
True
True


In [63]:
"""Tests for QKRR"""

import pytest
import numpy as np
import sympy as sp

from squlearn import Executor
from squlearn.encoding_circuit import LayeredEncodingCircuit
from squlearn.kernel.matrix import ProjectedQuantumKernel

class TestProjectedQuantumKernel:
    
    @pytest.fixture
    def setup_single_variable(self):
        x, y, gamma_sp, p = sp.symbols("x y gamma p")
        sympy_K = sp.exp(-2 * gamma_sp * (1 - sp.cos(x - y)) * sp.cos(p)**2)
        
        sympy_values = {
            "K": sympy_K,
            "dKdx": sp.diff(sympy_K, x),
            "dKdy": sp.diff(sympy_K, y),
            "dKdxdx": sp.diff(sp.diff(sympy_K, x), x)
        }
        
        return x, y, gamma_sp, p, sympy_values
    
    @pytest.fixture
    def setup_multi_variable(self):
        x0, y0, gamma_sp, p0 = sp.symbols("x0 y0 gamma p0")
        x1, y1, p1 = sp.symbols("x1 y1 p1")
        
        sympy_K = sp.exp(
            -2.0 * gamma_sp * (
                -sp.cos(p0)**2 * sp.cos(x0 - y0) + sp.cos(p0)**2 - 
                sp.cos(p1)**2 * sp.cos(x1 - y1) + sp.cos(p1)**2
            )
        )
        
        sympy_values = {
            "K": sympy_K,
            "dKdx0": sp.diff(sympy_K, x0),
            "dKdx1": sp.diff(sympy_K, x1),
            "dKdy0": sp.diff(sympy_K, y0),
            "dKdy1": sp.diff(sympy_K, y1),
            "dKdp0": sp.diff(sympy_K, p0),
            "dKdp1": sp.diff(sympy_K, p1),
        }
        
        return x0, y0, x1, y1, gamma_sp, p0, p1, sympy_values

    def test_single_variable_derivatives(self, setup_single_variable):
        x, y, gamma_sp, p, sympy_values = setup_single_variable
        
        x_num, y_num = 0.79, -0.31
        p_num = -0.63
        gamma_num = 0.08
        
        subs = {x: x_num, y: y_num, gamma_sp: gamma_num, p: p_num}
        sympy_num_values = {key: sympy_values[key].evalf(subs=subs) for key in sympy_values}
        
        executor = Executor()
        num_features = 1
        encoding_circuit = LayeredEncodingCircuit(num_qubits=num_features, num_features=num_features)
        encoding_circuit.Ry("p")
        encoding_circuit.Rx("x")
        
        kernel = ProjectedQuantumKernel(
            encoding_circuit=encoding_circuit, executor=executor, 
            outer_kernel="gaussian", gamma=gamma_num, initial_parameters=np.array([p_num])
        )
        
        values = kernel.evaluate_derivatives(
            [x_num], [y_num], ["K", "dKdx", "dKdy", "dKdxdx", "dKdp"]
        )
        
        for key in ["K", "dKdx", "dKdy", "dKdxdx"]:
            assert np.isclose(
                float(values[key][0][0]), float(sympy_num_values[key]), atol=1e-7
            ), f"Mismatch in {key}: {values[key][0][0]} vs {sympy_num_values[key]}"
    
    def test_multi_variable_derivatives(self, setup_multi_variable):
        x0, y0, x1, y1, gamma_sp, p0, p1, sympy_values = setup_multi_variable
        
        x0_num, y0_num = 0.79, -0.31
        x1_num, y1_num = 0.9, -1.31
        p_num = -0.63
        gamma_num = 0.08
        
        subs = {x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num, p1: p_num}
        sympy_num_values = {
            "K": sympy_values["K"].evalf(subs=subs),
            "dKdx": [sympy_values["dKdx0"].evalf(subs=subs), sympy_values["dKdx1"].evalf(subs=subs)],
            "dKdy": [sympy_values["dKdy0"].evalf(subs=subs), sympy_values["dKdy1"].evalf(subs=subs)],
            "dKdp": [sympy_values["dKdp0"].evalf(subs=subs), sympy_values["dKdp1"].evalf(subs=subs)],
        }
        
        executor = Executor()
        num_features = 2
        encoding_circuit = LayeredEncodingCircuit(num_qubits=num_features, num_features=num_features)
        encoding_circuit.Ry("p")
        encoding_circuit.Rx("x")
        
        kernel = ProjectedQuantumKernel(
            encoding_circuit=encoding_circuit, executor=executor, 
            outer_kernel="gaussian", gamma=gamma_num, initial_parameters=np.array([p_num, p_num])
        )
        
        values = kernel.evaluate_derivatives(
            [[x0_num, x1_num]], [[y0_num, y1_num]], ["K", "dKdx", "dKdy", "dKdp"]
        )
        
        for key in ["K", "dKdx", "dKdy", "dKdp"]:
            np_values = np.array(values[key]).flatten().astype(float)
            sympy_values_arr = np.array(sympy_num_values[key]).astype(float)
            assert np.allclose(np_values, sympy_values_arr, atol=1e-7), f"Mismatch in {key}: {np_values} vs {sympy_values_arr}"


In [31]:
np.allclose(v["K"].flatten() , v["K"].flatten())

True

In [ ]:
#  def matrix_eval_sympy(expression, X, Y):
#                 result = np.zeros((len(X), len(Y)))
#                 for i in range(len(X)):
#                     for j in range(len(Y)):
#                         result[i, j] = expression.evalf(subs={x0: X[i], y0: Y[j], x1: X[i], y1: Y[j], gamma_sp: gamma_num, p0: p_num})
#                 return result

                
            
#             #sympy_K.evalf(subs={x0: x0_num, y0: y0_num, x1: x1_num, y1: y1_num, gamma_sp: gamma_num, p0: p_num}),
#             sympy_values = {
#                 "K": matrix_eval_sympy(sympy_K, [x0_num, x1_num], [x0_num, x1_num]),